In [2]:
# import packages
import pandas as pd
import numpy as np

In [177]:
# import population by county data
# https://www2.census.gov/programs-surveys/popest/datasets/2020-2021/counties/totals/co-est2021-alldata.csv
# average us household size in 2021 = 3.13 
# https://www.statista.com/statistics/183657/average-size-of-a-family-in-the-us/

population = pd.read_csv('../data/input/pop_county.csv',
                         encoding='latin-1',
                         usecols=['STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'POPESTIMATE2021'],
                         dtype={'STATE': str, 'COUNTY': str}
                        )

# lowercase for column names
population.columns = population.columns.str.lower()                        

In [179]:
# define countyCode and no_households
population['countyCode'] = population.state + population.county 
population['no_hh_2021'] = population.popestimate2021/3.13 # divide population of county by average US household (2021)

# mapping of us states to abbreviations
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

# apply mapping
population['state_2d'] = population.stname.map(us_state_to_abbrev)

# subset data
population = population[['countyCode', 'ctyname', 'state_2d', 'popestimate2021', 'no_hh_2021']]

In [180]:
# drop rows where ctyname is a state
population = population.drop(population[~population.ctyname.str.contains('County')].index).reset_index(drop=True)

In [181]:
# final population
population

,countyCode,ctyname,state_2d,popestimate2021,no_hh_2021
0,01001,Autauga County,AL,59095,18880.191693
1,01003,Baldwin County,AL,239294,76451.757188
2,01005,Barbour County,AL,24964,7975.718850
3,01007,Bibb County,AL,22477,7181.150160
4,01009,Blount County,AL,59041,18862.939297
...,...,...,...,...,...
3002,56037,Sweetwater County,WY,41614,13295.207668
3003,56039,Teton County,WY,23575,7531.948882
3004,56041,Uinta County,WY,20635,6592.651757
3005,56043,Washakie County,WY,7705,2461.661342


In [182]:
# import house price data
# median house price by county in q4 2021
# https://www.nar.realtor/research-and-statistics/housing-statistics/county-median-home-prices-and-monthly-mortgage-payment

house_price = pd.read_csv('../data/input/houseprice.csv', 
                          encoding='utf-16', 
                          sep='\t',
                          dtype={'Full County Number': str}
                         )


In [183]:
house_price.columns = house_price.columns.str.replace(' ','_').str.lower()
house_price = house_price.rename({'q4_2021': 'med_house_price_2021',
                                  'full_county_number': 'countyCode'}, 
                                  axis=1
                                )

In [184]:
house_price.med_house_price_2021 = house_price.med_house_price_2021.str[1:].str.replace(',','')
house_price.med_house_price_2021 = house_price.med_house_price_2021.astype('int64', errors='ignore')

In [185]:
house_price.dropna(axis=0)

,countyCode,med_house_price_2021
0,55107,139179
1,55099,150348
2,55078,124942
3,55067,139793
4,55003,138811
...,...,...
3115,06075,1230797
3116,06041,1090583
3121,46095,39349
3122,46017,60066


In [186]:
# import latitude and longitude data
# data on county, average latitude and longitude data for centre of county
# https://gist.github.com/russellsamora/12be4f9f574e92413ea3f92ce1bc58e6

lat_long = pd.read_csv('../data/input/county_lat_long.csv', 
                       encoding='latin-1',
                       usecols=['fips_code', 'lat', 'lng'],
                       dtype={'fips_code': str}
                      )

In [201]:
lat_long

,fips_code,lng,lat
0,01059,-87.843283,34.442381
1,13111,-84.319296,34.864126
2,19109,-94.206898,43.204140
3,40115,-94.810589,36.835878
4,42115,-75.800905,41.821277
...,...,...,...
3228,12029,-83.158705,29.608068
3229,18017,-86.346207,40.761660
3230,26091,-84.066412,41.894694
3231,72003,-67.175247,18.360392


In [203]:
lat_long.rename({'fips_code': 'countyCode'}, axis=1, inplace=True)

In [222]:
# import household income data
# data on median household income by county 
# https://www.ers.usda.gov/data-products/county-level-data-sets/county-level-data-sets-download-data/

med_inc_hh_2020 = pd.read_csv('../data/input/income.csv', 
                              usecols=['FIPS_Code', 'Attribute', 'Value'],
                              dtype={'FIPS_Code': str}
                             )

In [223]:
med_inc_hh_2020 = med_inc_hh_2020[med_inc_hh_2020['Attribute'] == 'Median_Household_Income_2020'].reset_index(drop=True)

In [224]:
# leading zero missing from 4-digit FIPS; add this back
def add_zero(x):
    if len(x) == 4:
        return '0' + x
    else:
        return x

med_inc_hh_2020['FIPS_Code'] = med_inc_hh_2020.FIPS_Code.apply(add_zero)

In [225]:
med_inc_hh_2020 = med_inc_hh_2020.rename({'FIPS_Code': 'countyCode', 'Value': 'med_HH_inc_2020'}, axis=1)
med_inc_hh_2020.drop('Attribute', axis=1, inplace=True)

In [226]:
# merge all datasets together
pop_house_price = pd.merge(left=population, right=house_price, on='countyCode', how='left')
pop_house_price_inc = pd.merge(left=pop_house_price, right=med_inc_hh_2020, on='countyCode', how='left')
county_chars_all = pd.merge(left=pop_house_price_inc, right=lat_long, on='countyCode', how='left')

In [227]:
county_chars_all.rename({'ctyname': 'countyName',
                         'state_2d': 'stateName', 
                         'popestimate2021': 'pop_2021',
                         'lng': 'long'},
                        axis=1, inplace=True
                        )

In [228]:
county_chars_all = county_chars_all.dropna(axis=0).reset_index(drop=True)

In [229]:
county_chars_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2992 entries, 0 to 2991
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   countyCode            2992 non-null   object 
 1   countyName            2992 non-null   object 
 2   stateName             2992 non-null   object 
 3   pop_2021              2992 non-null   int64  
 4   no_hh_2021            2992 non-null   float64
 5   med_house_price_2021  2992 non-null   object 
 6   med_HH_inc_2020       2992 non-null   float64
 7   long                  2992 non-null   float64
 8   lat                   2992 non-null   float64
dtypes: float64(4), int64(1), object(4)
memory usage: 210.5+ KB


In [232]:
county_chars_all['pop_2021'] = county_chars_all['pop_2021'].astype('float')
county_chars_all['med_house_price_2021'] = county_chars_all['med_house_price_2021'].astype('float')

In [233]:
county_chars_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2992 entries, 0 to 2991
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   countyCode            2992 non-null   object 
 1   countyName            2992 non-null   object 
 2   stateName             2992 non-null   object 
 3   pop_2021              2992 non-null   float64
 4   no_hh_2021            2992 non-null   float64
 5   med_house_price_2021  2992 non-null   float64
 6   med_HH_inc_2020       2992 non-null   float64
 7   long                  2992 non-null   float64
 8   lat                   2992 non-null   float64
dtypes: float64(6), object(3)
memory usage: 210.5+ KB


In [235]:
# save dataframe as parquet to preserve datatypes
county_chars_all.to_parquet('../data/output/county_chars_all.gzip', compression='gzip')